# A Demonstration of the Diffie-Hellman Key Agreement Protocol

The Diffie-Hellman protocol presented below simulates a confidential message exchange between two parties (Alice and Bob) over an insecure channel. Further, a third party (Eve), who intercepts the message encrypted using this protocol on the insecure channel, has no feasible way of learning its content.

Diffie and Hellman first introduced this protocol in 1976, in their seminal paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf). In it, the authors proposed a novel cryptosystem in which a publicly-known key could be used to encrypt a message that only the holder of a private key could decrypt, thereby solving the age-old problem of secure key-exchange in cryptographic schemes.

The method is based on a one-way function that pits the difficulty of solving the *discrete logarithm problem* (or *DLP*) in large-order multiplicative integer groups against the ease of exponentiation (aka repeated multiplication) of integers in such groups.

## The Source Code

The following sequence of function calls to the `dh` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice encrypts a private message, and then transmits it to Bob over an insecure channel. Meanwhile, Eve&mdash;a passive adversary&mdash;intercepts the message transmitted on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `dh` module; but that only Alice and Bob possess their own private keys, and also a shared encryption key each generates using their private keys as input. Eve, not having access to either Alice's or Bob's private keys, therefore cannot feasibly derive the session key Alice uses to encrypt the message she sends to Bob, thereby preventing Eve from learning the contents of the message.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must not be transmitted over the insecure channel).

Textual descriptions of behavior appear below the code snippets and their outputs.

In [1]:
import sys
sys.path.append("../code/src")
import dh
sys.path.append("../code/test")
import sym

Import the `dh` module, and a module `sym` that implements an elementary symmetric cipher that Alice and Bob will use in the session.

In [2]:
q, p, g = dh.generate_parameters(dh.min_p_bit_len)

The protocol begins with Alice computing the public domain parameters both she and Bob will use to set up a session in which Alice transmits an encrypted message to Bob.

The argument passed to the function `dh.generate_parameters(dh.min_p_bit_len)` specifies the size, in bits, of the public modulus to be used in the protocol; the security of the system is directly proportional to the size of this modulus (`dh.min_p_bit_len` is an integer constant whose value is `2048`).

The parameter *[p]* returned by `dh.generate_parameters()` is the public modulus (a prime number). *[q]* (also a prime number) is the size of the smallest subgroup modulo *[p]*; the subgroup within which the public keys (to be generated in a subsequent step) must fall. And *[g]* is the generator of this subgroup.

In [3]:
print(p.bit_length())
print(q.bit_length())

2048
256


Here, we print the bit lengths of the first two parameters returned by `dh.generate_parameters()`. Note that the length of *[p]*, the public modulus, is in fact the `2048` bits requested. The bit length of *[q]*&mdash;`256`&mdash;represents the size of smallest possible subgroup of the full group modulo *[p]*.

We are not particularly interested in the bit-length of the generator *[g]*, but rather that it will generate the entire subgroup represented by *[q]*. We prove this by calling the function `dh._validate_parameters(q, p, g)`.

In [4]:
dh._validate_parameters(q, p, g)

If this function does not a raise an exception, we know the parameters are suitable to generate keys to be used for a secure message exchange.

In [5]:
kA, KA = dh.generate_keypair(q, p, g)

Alice now computes a key pair; *kA* is her private key, and *[KA]* her public key (the keys are suffixed with the captital letter *A* to indicate that they belong to Alice).

Alice transmits *[p]*, *[q]*, *[g]* and *[KA]* to Bob, whereupon Bob calls the same function Alice did above to validate the public parameters, and another function to validate Alice's public key *[KA]*.

In [6]:
dh._validate_parameters(q, p, g)
dh.validate_pub_key(KA, q, p)

If neither function raises an exception, Bob knows that all the public parameters and Alice's public key are valid, and the session is allowed to continue.

In [7]:
kB, KB = dh.generate_keypair(q, p, g)

Using the public parameters sent to him by Alice, Bob now computes a key pair of his own, storing them in the variables *kB* and *[KB]*.

Bob transmits his public key *[KB]* to Alice.

In [8]:
dh.validate_pub_key(KB, q, p)

On receiving Bob's public key *[KB]*, Alice must validate it.

In [9]:
kSessionA = dh.generate_session_key(KB, kA, p)

Now Alice computes the session key that will be used to encrypt the message she will transmit to Bob. She calls the function `dh.generate_session_key()`, passing it Bob's public key *[KB]*, Alice's private key *kA* and the public modulus *p*.

Basically, all this function does is raise Bob's public key *[KB]* to the power of Alice's private key *kA* modulo the public modulus *p*.

In [10]:
kSessionB = dh.generate_session_key(KA, kB, p)

Now Bob calls the same function Alice did to compute a session key of his own, only in Bob's case he passes Alice's public key *[KA]* and his private key *kB* to `dh.generate_session_key`.

In [11]:
print(kSessionA == kSessionB)

True


If the essential property of DH holds, both keys&mdash;*kSessionA* and *kSessionB*&mdash;must be equivalent.

In [12]:
mA = "Encrypt me!"

Now, Alice composes a message for Bob, and stores it in the variable *mA*.

In [13]:
mAC = sym.encrypt(kSessionA, mA)

Next, Alice encrypts the message, supplying it and her session key to the function `sym.encrypt()`, and stores the ciphertext in *[mAC]*.

Alice is now ready to transmit the ciphertext to Bob over the insecure channel, which she does.

In [14]:
mB = sym.decrypt(kSessionB, mAC)

Bob, having received the ciphertext *[mAC]* from Alice, decrypts it by calling the function `sym.decrypt()`, passing it his session key *kSessionB* and the ciphertext *[mAC]*, and stores the result in *mB*.

In [15]:
print(mB)

Encrypt me!


Here, we print *mB*, thereby proving that it matches the plaintext *mA* that Alice encrypted and transmitted to Bob.